# Tuto spisem, co vsetko mam
- Explorativna analyza
- Agresivita samce samice, ale celkom dobre
- Kastracia
- Kastracia pokracovanie
- Kastracia pokracovanie
- Agresivita psov vs vek (este doplnit brechanie treba)
- Impulzivita a Nepozornost vs vek
- Strach vs vek
- Najtazsie a najlahsie plemena (mohol by som skusit spravit nejaku zavislost na agresivite v zavislosti od velkosti plemena)
- Agresivita vs Strach - zaujimave
- Agresivita vs citlivost na hluk
- Strach vs citlivost na hluk

### Napady
- 

# Napady na hypotezy
- v suvislosti oddelenia od matky, bude psom najprv stupat a potom klesat, nebojacnost a veselost (vitalita)
### Najprv zistenia
- Samci su agresivnejsi k vlastnemmu pohlaviu, a samice k opacnemu, ale iba o trochu, samci su agresivnejsi k ludom


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from IPython.display import Markdown
# from google.colab import files
# uploaded = files.upload()

### Prikazy na stiahnutie z excelu a ulozenie do csv kvoli nacitavaniu
# dog_info = pd.read_excel("data-for-publication.xlsx", sheet_name="Information", header=0)
# dog_info.to_csv("psy_info.csv")
# files.download("psy_info.csv")

## Stiahnem data a ulozim ich v csv a vyrobim dataframe dogs a dogs info

In [ ]:
dogs = pd.read_csv("psy_data.csv", header=0).set_index("dog.id")
dog_info = pd.read_csv("psy_info.csv", header=0)
casti = ['Demographic information', 'Environment and background', 'Personality', 
     'Noise sensitivity', 'Fearfulness', 'Aggression', 
     'Fear of surfaces and heights', 'Separation anxiety', 'Impulsivity/ inattention']
zoz = []
for i in dog_info['SECTION']:
    if i in casti:
        slovo = i
    zoz.append(slovo)
dog_info['SECTION'] = pd.Series(zoz)
dog_info = dog_info.set_index('SECTION').drop('Unnamed: 0', axis=1)


## Tuto budu zoznamy jednotlivych casti, a urobene casti tabulky podla kategorie


In [ ]:
spojite_vsetky = dog_info[dog_info['COLUMN VALUES'].str.contains('continuous', na=False)]
spojite = list(spojite_vsetky.query('SECTION != "Demographic information"')['COLUMN NAME'])
spojite_set = set(spojite)


dogs_male = dogs.query("sex=='male'")
dogs_female = dogs.query("sex=='female'")

dogs_demographic = dogs.transpose().iloc[0:11].transpose()
dogs_environment = dogs.transpose().iloc[11:42].transpose()
dogs_personality = dogs.transpose().iloc[42:112].transpose()
dogs_noise_sensitivity = dogs.transpose().iloc[112:154].transpose()
dogs_fearfullness = dogs.transpose().iloc[154:189].transpose()
dogs_aggression = dogs.transpose().iloc[189:214].transpose()
dogs_fear_surfaces = dogs.transpose().iloc[214:222].transpose()
dogs_separation_anxiety = dogs.transpose().iloc[222:232].transpose()
dogs_inattention = dogs.transpose().iloc[232:250].transpose()


demographic = list(dogs_demographic.columns)
environment = list(dogs_environment.columns)
personality = list(dogs_personality.columns)
noise_sensitivity = list(dogs_noise_sensitivity.columns)
fearfullness = list(dogs_fearfullness.columns)
aggression = list(dogs_aggression.columns)
fear_surfaces = list(dogs_fear_surfaces.columns)
separation_anxiety = list(dogs_separation_anxiety.columns)
inattention = list(dogs_inattention.columns)

demographic_continuous = list(spojite_set & set(demographic))
environment_continuous = list(spojite_set & set(environment))
personality_continuous = list(spojite_set & set(personality))
noise_sensitivity_continuous = list(spojite_set & set(noise_sensitivity))
fearfullness_continuous = list(spojite_set & set(fearfullness))
aggression_continuous = list(spojite_set & set(aggression))
fear_surfaces_continuous = list(spojite_set & set(fear_surfaces))
separation_anxiety_continuous = list(spojite_set & set(separation_anxiety))
inattention_continuous = list(spojite_set & set(inattention))

demographic_discrete = list(set(demographic) - spojite_set)
environment_discrete = list(set(environment) - spojite_set)
personality_discrete = list(set(personality) - spojite_set)
noise_sensitivity_discrete = list(set(noise_sensitivity) - spojite_set)
fearfullness_discrete = list(set(fearfullness) - spojite_set)
aggression_discrete = list(set(aggression) - spojite_set)
fear_surfaces_discrete = list(set(fear_surfaces) - spojite_set)
separation_anxiety_discrete = list(set(separation_anxiety) - spojite_set)
inattention_discrete = list(set(inattention) - spojite_set)

nespojite = environment_discrete+personality_discrete+noise_sensitivity_discrete+fearfullness_discrete+aggression_discrete+fear_surfaces_discrete+separation_anxiety_discrete+inattention_discrete
nespojite = list(set(nespojite)-set(environment))

# Explorativna analyza

In [ ]:
breeds = list(dogs['breed_group'].unique())
breed_count = len(breeds)
male_count = len(dogs.query("sex=='male'"))
female_count = len(dogs.query("sex=='female'"))

# print('plemena:', breeds)
print('mnozstvo samcov:', male_count)
print('mnozstvo samic:', female_count)
print('nozstvo plemien:', breed_count)


figure, axes = plt.subplots(1, 1, figsize=(13, 10))
sns.violinplot(data=dogs[spojite], orient='h', ax=axes)


## Agresivita
#### samce vs samice

In [ ]:

figure, axes = plt.subplots(1, 1, figsize=(6, 5), sharey=True)

popisy = {
    'Aggressive_dogs_samegender': 'Voči psom rovnakého pohlavia',
    'Aggressive_dogs_oppositegender': 'Voči psom opačného pohlavia',
    'Aggressive_people': 'Voči ľuďom',
    'aggressiveness_dominance_score': 'Dominancia / Agresivita',
    'stranger_aggression_score': 'Agresivita voči cudzim',
    'owner_aggression_score': 'Agresivita voči majiteľovi',
    'dog_aggression_score': 'Agresivita voči psom',
    'barking_score': 'Brechanie'
}

agresivita_gender = dogs.groupby('sex')[['Aggressive_dogs_samegender', 'Aggressive_dogs_oppositegender', 'Aggressive_people']].mean().reset_index().melt(id_vars='sex')
agresivita_gender['variable'] = agresivita_gender['variable'].replace(popisy)
sns.barplot(agresivita_gender, y='variable', x='value', hue='sex', palette={'male':'blue', 'female':'violet'}, ax=axes, orient='h')


axes.set_title('Priemerná agresivita', fontsize=14)
axes.set_xlabel('Priemerná hodnota', fontsize=12)
axes.set_ylabel('Typ agresivity', fontsize=12)
axes.legend(title='Pohlavie')
plt.tight_layout()
plt.show()


figure, axes = plt.subplots(1, 1)
plt.xticks(ha='right') 
agresivita_gender = dogs.groupby('sex')[['aggressiveness_dominance_score', 'stranger_aggression_score', 'owner_aggression_score', 'dog_aggression_score', 'barking_score']].mean().reset_index().melt(id_vars='sex')
agresivita_gender['variable'] = agresivita_gender['variable'].replace(popisy)

sns.barplot(agresivita_gender, x='variable', y='value', hue='sex', palette={'male':'blue', 'female':'violet'})
axes.tick_params(axis='x', rotation=25)

axes.set_title('Priemerná agresivita', fontsize=14)
axes.set_ylabel('Priemerná hodnota', fontsize=12)
# axes.set_xlabel('Typ agresivity', fontsize=12)
axes.legend(title='Pohlavie')
plt.tight_layout()
plt.show()


# Kastracia
zistovanie najrozdielnejsich vlastnosti v ramci kastracie

In [ ]:
# pocet = 10
# pripocet = 1

# kastracia_male = dogs.query('sex=="male"').groupby("sterilization_status").mean(numeric_only=True).transpose().iloc[8:]
# kastracia_male.columns.name = None
# kastracia_male["personality_trait"] = kastracia_male.index
# display(kastracia_male)

# kastracia_rozdiely_male = kastracia_male.copy(deep=True)
# kastracia_rozdiely_male["diff"] = abs(kastracia_rozdiely_male["intact"] - kastracia_rozdiely_male["sterilized"])
# kastracia_rozdiely_male["relative_diff"] = abs((kastracia_rozdiely_male["intact"] - kastracia_rozdiely_male["sterilized"])/(abs(kastracia_rozdiely_male["intact"])+abs(kastracia_rozdiely_male["sterilized"])+pripocet))

# index_male_diff = list(kastracia_rozdiely_male.sort_values('diff', ascending=False).head(pocet).index)
# index_male_rel_diff = list(kastracia_rozdiely_male.sort_values('relative_diff', ascending=False).head(pocet).index)

# ###############################################################################################################################

# kastracia_female = dogs.query('sex=="female"').groupby("sterilization_status").mean(numeric_only=True).transpose().iloc[8:]
# kastracia_female.columns.name = None
# kastracia_female["personality_trait"] = kastracia_female.index

# kastracia_rozdiely_female = kastracia_female.copy(deep=True)
# kastracia_rozdiely_female["diff"] = abs(kastracia_rozdiely_female["intact"] - kastracia_rozdiely_female["sterilized"])
# kastracia_rozdiely_female["relative_diff"] = abs((kastracia_rozdiely_female["intact"] - kastracia_rozdiely_female["sterilized"])/(abs(kastracia_rozdiely_female["intact"])+abs(kastracia_rozdiely_female["sterilized"])+pripocet))


# index_female_diff = list(kastracia_rozdiely_female.sort_values('diff', ascending=False).head(pocet).index)
# index_female_rel_diff = list(kastracia_rozdiely_female.sort_values('relative_diff', ascending=False).head(pocet).index)


In [ ]:
pocet = 10
pripocet = 1

kastracia_male = dogs.query('sex=="male"').groupby("sterilization_status").mean(numeric_only=True).transpose().iloc[8:]
kastracia_male.columns.name = None
kastracia_male["personality_trait"] = kastracia_male.index

kastracia_rozdiely_male_nespojite = kastracia_male.loc[nespojite, :].copy(deep=True)
kastracia_rozdiely_male_nespojite["diff"] = abs(kastracia_rozdiely_male_nespojite["intact"] - kastracia_rozdiely_male_nespojite["sterilized"])
index_male_nespojite = list(kastracia_rozdiely_male_nespojite.sort_values('diff', ascending=False).head(pocet).index)


kastracia_rozdiely_male_spojite = kastracia_male.loc[spojite, :].copy(deep=True)
kastracia_rozdiely_male_spojite["diff"] = abs(kastracia_rozdiely_male_spojite["intact"] - kastracia_rozdiely_male_spojite["sterilized"])
index_male_spojite= list(kastracia_rozdiely_male_spojite.sort_values('diff', ascending=False).head(pocet).index)

###################################################################################################################################

kastracia_female = dogs.query('sex=="female"').groupby("sterilization_status").mean(numeric_only=True).transpose().iloc[8:]
kastracia_female.columns.name = None
kastracia_female["personality_trait"] = kastracia_female.index

kastracia_rozdiely_female_nespojite = kastracia_female.loc[nespojite, :].copy(deep=True)
kastracia_rozdiely_female_nespojite["diff"] = abs(kastracia_rozdiely_female_nespojite["intact"] - kastracia_rozdiely_female_nespojite["sterilized"])
index_female_nespojite = list(kastracia_rozdiely_female_nespojite.sort_values('diff', ascending=False).head(pocet).index)


kastracia_rozdiely_female_spojite = kastracia_female.loc[spojite, :].copy(deep=True)
kastracia_rozdiely_female_spojite["diff"] = abs(kastracia_rozdiely_female_spojite["intact"] - kastracia_rozdiely_female_spojite["sterilized"])
index_female_spojite= list(kastracia_rozdiely_female_spojite.sort_values('diff', ascending=False).head(pocet).index)


## Kastracia pokracovanie - Grafy
vygrafovanie zistenych vlastnosti

In [ ]:
dogs_melted_male_spojite = kastracia_male.loc[index_male_spojite, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")

dogs_melted_female_spojite = kastracia_female.loc[index_female_spojite, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")
palette = {'sterilized': 'dodgerblue', 'intact': 'tomato'}



figure, axes = plt.subplots(1, 2, figsize=(21, 7), sharey=True)
sns.barplot(data=dogs_melted_male_spojite, x=dogs_melted_male_spojite.index, y="value", hue="sterilization_status", ax=axes[0], palette=palette)
axes[0].set_xticklabels(axes[0].get_xticklabels(), ha='right')
axes[0].tick_params(axis='x', rotation=30)
axes[0].set_title("male")
sns.barplot(data=dogs_melted_female_spojite, x=dogs_melted_female_spojite.index, y="value", hue="sterilization_status", ax=axes[1], palette=palette)
axes[1].set_xticklabels(axes[1].get_xticklabels(), ha='right')
axes[1].tick_params(axis='x', rotation=30)
axes[1].set_title("female")

###########################################

dogs_melted_male_nespojite = kastracia_male.loc[index_male_nespojite, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")
dogs_melted_female_nespojite = kastracia_female.loc[index_female_nespojite, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")

figure, axes = plt.subplots(1, 2, figsize=(26, 7), sharey=True)
for ax in axes:
    ax.axhline(0, color='gray', linestyle='--', linewidth=1)

sns.barplot(data=dogs_melted_male_nespojite, x=dogs_melted_male_nespojite.index, y="value", hue="sterilization_status", ax=axes[0], palette=palette)
plt.xticks(ha='right') 
axes[0].tick_params(axis='x', rotation=30)
axes[0].set_xticklabels(axes[0].get_xticklabels(), ha='right')
axes[0].set_title("male")
sns.barplot(data=dogs_melted_female_nespojite, x=dogs_melted_female_nespojite.index, y="value", hue="sterilization_status", ax=axes[1], palette=palette)
axes[1].set_xticklabels(axes[1].get_xticklabels(), ha='right')
axes[1].tick_params(axis='x', rotation=30)
axes[1].set_title("female")

# TUTO POKRACUJEM RANO

## Kastracia dalsia
planujem urobit ze zoberiem spojite premenne z jednotlivych casti dotaznika
a porovnam s ktorou suvisi kastrovanie najviac (nemam nicim podlozene, ale pride mi ze pri jednotlivych otazkach to moze byt aj trochu nahodne)

In [ ]:
dogs_melted_spojite_male = kastracia_male.loc[spojite, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")
dogs_melted_spojite_female = kastracia_female.loc[spojite, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")


figure, axes = plt.subplots(1, 2, figsize=(14, 7))
sns.barplot(data=dogs_melted_spojite_male, y=dogs_melted_spojite_male.index, x="value", hue="sterilization_status", ax=axes[0], orient='h')
# axes[0].tick_params(axis='y', rotation=30)
axes[0].set_title("male")
# axes[0].set_xticklabels(axes[0].get_xticklabels(), ha='right')
sns.barplot(data=dogs_melted_spojite_female, y=dogs_melted_spojite_female.index, x="value", hue="sterilization_status", ax=axes[1], orient='h')
# axes[1].tick_params(axis='y', rotation=30)
axes[1].set_title("female")
# axes[1].set_xticklabels(axes[1].get_xticklabels(), ha='right')

## Kastracia a agresivita
porovnam podobnym bargrafom agresivitu

In [ ]:
dogs_melted_aggression_male = kastracia_male.loc[aggression_discrete, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")
dogs_melted_aggression_female = kastracia_female.loc[aggression_discrete, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")

figure, axes = plt.subplots(1, 2, figsize=(9, 8), sharey=True)
sns.barplot(data=dogs_melted_aggression_male, y=dogs_melted_aggression_male.index, x="value", hue="sterilization_status", ax=axes[0], orient='h')
# axes[0].tick_params(axis='x', rotation=30)
axes[0].set_title("male")
sns.barplot(data=dogs_melted_aggression_female, y=dogs_melted_aggression_female.index, x="value", hue="sterilization_status", ax=axes[1], orient='h')
# axes[1].tick_params(axis='x', rotation=30)
axes[1].set_title("female")


## Kastracia dalsia


In [ ]:
dogs_melted_personality_male = kastracia_male.loc[personality_continuous, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")
dogs_melted_personality_female = kastracia_female.loc[personality_continuous, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")

figure, axes = plt.subplots(1, 2, figsize=(9, 4), sharey=True)
sns.barplot(data=dogs_melted_personality_male, y=dogs_melted_personality_male.index, x="value", hue="sterilization_status", ax=axes[0], orient='h')
# axes[0].tick_params(axis='x', rotation=30)
axes[0].set_title("male")
sns.barplot(data=dogs_melted_personality_female, y=dogs_melted_personality_female.index, x="value", hue="sterilization_status", ax=axes[1], orient='h')
# axes[1].tick_params(axis='x', rotation=30)
axes[1].set_title("female")

figure, axes1 = plt.subplots(1, 2, figsize=(9, 4), sharey=True)

sns.violinplot(dogs.query('sex=="male"')[personality_continuous], ax=axes1[0], orient='h')
sns.violinplot(dogs.query('sex=="female"')[personality_continuous], ax=axes1[1], orient='h')


### Tuto je agresivita psov
##### ku psom, majitelom a cudzncom
podla veku 

In [ ]:
psy1 = dogs.copy(deep=True)
psy1["Aggression_age"] = psy1["Aggression_age"].round(1)

psy1 = psy1.groupby("Aggression_age")[aggression_continuous].mean()

fig, axes = plt.subplots(1, 1, figsize=(6, 4))
sns.histplot(data=dogs, x="Aggression_age", ax=axes)
axes.set_title("Histogram veku vypĺňania dotazníka", fontsize=13)
axes.set_xlabel("Vek (roky)", fontsize=12)
axes.set_ylabel("Počet psov", fontsize=12)
plt.tight_layout()
plt.show()

fig, axes = plt.subplots(1, 3, figsize=(16, 5), sharey=True)

sns.scatterplot(x=psy1.index, y=psy1["stranger_aggression_score"], ax=axes[0])
axes[0].set_title("Voči cudzincom")
axes[0].set_xlabel("Vek (roky)")
axes[0].set_ylabel("Agresia")

sns.scatterplot(x=psy1.index, y=psy1["owner_aggression_score"], ax=axes[1])
axes[1].set_title("Voči majiteľovi")
axes[1].set_xlabel("Vek (roky)")

sns.scatterplot(x=psy1.index, y=psy1["dog_aggression_score"], ax=axes[2])
axes[2].set_title("Voči psom")
axes[2].set_xlabel("Vek (roky)")

plt.suptitle("Typy agresie podľa veku psa", fontsize=14)
plt.tight_layout()
plt.show()


### Teraz spravim podobne impulzivitu a nepozornost
nasiel som jemnu zavislost, impulzivita stupa a vrcholi okolo 1 roka, potom pomaly klesa, nepozornost je na tom velmi podobne



In [ ]:
psy2 = dogs.copy(deep=True)
psy2["Impulsivity_age"] = psy2["Impulsivity_age"].round(1)
psy2 = psy2.groupby("Impulsivity_age")[inattention_continuous].mean()


ax = sns.histplot(data=dogs, x="Impulsivity_age")

ax.set_title("Histogram veku pri vypĺňaní impulzivity")
ax.set_xlabel("Vek (roky)")
ax.set_ylabel("Počet psov")

fig, axes = plt.subplots(1, 2, figsize=(16, 5), sharey=True)

sns.scatterplot(x=psy2.index, y=psy2["inattention_score"], ax=axes[0])
sns.regplot(x=psy2.index, y="inattention_score", data=psy2, ax=axes[0], scatter=True, line_kws={"color": "red"})
axes[0].set_title("Nepozornosť")
axes[0].set_xlabel("Vek (roky)")
axes[0].set_ylabel("Skóre")

sns.scatterplot(x=psy2.index, y=psy2["impulsivity_score"], ax=axes[1])
sns.regplot(x=psy2.index, y="impulsivity_score", data=psy2, ax=axes[1], scatter=True, line_kws={"color": "red"})


axes[1].set_title("Impulzivita")
axes[1].set_xlabel("Vek (roky)")
axes[1].set_ylabel("Skóre")

plt.suptitle("Nepozornosť a impulzivita podľa veku psa", fontsize=14)
plt.tight_layout()
plt.show()

### Teraz spravim podobne strach
pri strachu ale nevidim ziadnu suvislost




In [ ]:
psy3 = dogs.copy(deep=True)
psy3["Fearfulness_age"] = psy3["Fearfulness_age"].round(1)
psy3 = psy3.groupby("Fearfulness_age")[fearfullness_continuous].mean()

ax = sns.histplot(data=dogs, x="Fearfulness_age")

ax.set_title("Histogram veku pri vypĺňaní strachu")
ax.set_xlabel("Vek (roky)")
ax.set_ylabel("Počet psov")

figure, axes = plt.subplots(1, 1, figsize=(6, 4), sharey=True)
sns.scatterplot(x=psy3.index, y="fearfulness_score", data=psy3, ax=axes)

axes.set_title("Strach podľa veku")
axes.set_xlabel("Vek (roky)")
axes.set_ylabel("Skóre strachu")

plt.tight_layout()
plt.show()



### Priemerne hmotnosti jednotlivych plemien barplot


In [ ]:

psy_hmostnost_max = pd.DataFrame(dogs.groupby('breed_group')['weight_kg'].mean().sort_values()).head()
psy_hmostnost_min = pd.DataFrame(dogs.groupby('breed_group')['weight_kg'].mean().sort_values()).tail()

figure, axes = plt.subplots(1, 2, figsize=(8, 3), sharey=True)
sns.barplot(data=psy_hmostnost_max, x=psy_hmostnost_max.index, y='weight_kg', ax=axes[0])
axes[0].tick_params(axis='x', rotation=45)
sns.barplot(data=psy_hmostnost_min, x=psy_hmostnost_min.index, y='weight_kg', ax=axes[1])
axes[1].tick_params(axis='x', rotation=45)


pocty_psov = pd.DataFrame(dogs.groupby("breed_group").count().sort_values(by="sex"))
# figure, axes1 = plt.subplots(1, 1)
# display(pocty_psov)
# sns.histplot(data=pocty_psov, ax=axes1)

### Teraz Spravim Agresivitu vs Strach

In [ ]:

psy4 = dogs.copy(deep=True)
psy4["fearfulness_score"] = psy4["fearfulness_score"].round(1)
psy4 = psy4.groupby("fearfulness_score")[aggression_continuous].mean()

# Histogram strachu
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
sns.histplot(data=dogs, x="fearfulness_score", ax=ax)
ax.set_title("Histogram skóre strachu")
ax.set_xlabel("Skóre strachu")
ax.set_ylabel("Počet psov")
plt.tight_layout()
plt.show()

# Scatter ploty agresivity vs. strach
fig, axes = plt.subplots(2, 2, figsize=(13, 9), sharey=True)

sns.scatterplot(x=psy4.index, y=psy4["stranger_aggression_score"], ax=axes[0][0])
axes[0][0].set_title("Agresia voči cudzincom")
axes[0][0].set_xlabel("Skóre strachu")
axes[0][0].set_ylabel("Agresia")

sns.scatterplot(x=psy4.index, y=psy4["owner_aggression_score"], ax=axes[0][1])
axes[0][1].set_title("Agresia voči majiteľovi")
axes[0][1].set_xlabel("Skóre strachu")
axes[0][1].set_ylabel("Agresia")

sns.scatterplot(x=psy4.index, y=psy4["dog_aggression_score"], ax=axes[1][0])
axes[1][0].set_title("Agresia voči psom")
axes[1][0].set_xlabel("Skóre strachu")
axes[1][0].set_ylabel("Agresia")

sns.scatterplot(x=psy4.index, y=psy4["barking_score"], ax=axes[1][1])
axes[1][1].set_title("Štekot")
axes[1][1].set_xlabel("Skóre strachu")
axes[1][1].set_ylabel("Agresia")

plt.suptitle("Vzťah medzi strachom a Agresivitou/štekaním", fontsize=14)
plt.tight_layout()
plt.show()



### Citlivost na Hluk vs Agresivita

In [ ]:
psy5 = dogs.copy(deep=True)
psy5["noise_sensitivity_score"] = psy5["noise_sensitivity_score"].round(1)
psy5 = psy5.groupby("noise_sensitivity_score")[aggression_continuous].mean()

# Histogram citlivosti na hluk
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
sns.histplot(data=dogs, x="noise_sensitivity_score", binwidth=0.1, ax=ax)
ax.set_title("Histogram citlivosti na hluk")
ax.set_xlabel("Skóre citlivosti na hluk")
ax.set_ylabel("Počet psov")
plt.tight_layout()
plt.show()

# Scatter ploty agresivity vs. citlivosť na hluk
fig, axes = plt.subplots(2, 2, figsize=(13, 9), sharey=True)

sns.scatterplot(x=psy5.index, y=psy5["stranger_aggression_score"], ax=axes[0][0])
axes[0][0].set_title("Agresia voči cudzincom podľa citlivosti na hluk")
axes[0][0].set_xlabel("Skóre citlivosti na hluk")
axes[0][0].set_ylabel("Agresia")

sns.scatterplot(x=psy5.index, y=psy5["owner_aggression_score"], ax=axes[0][1])
axes[0][1].set_title("Agresia voči majiteľovi podľa citlivosti na hluk")
axes[0][1].set_xlabel("Skóre citlivosti na hluk")
axes[0][1].set_ylabel("Agresia")

sns.scatterplot(x=psy5.index, y=psy5["dog_aggression_score"], ax=axes[1][0])
axes[1][0].set_title("Agresia voči psom podľa citlivosti na hluk")
axes[1][0].set_xlabel("Skóre citlivosti na hluk")
axes[1][0].set_ylabel("Agresia")

sns.scatterplot(x=psy5.index, y=psy5["barking_score"], ax=axes[1][1])
axes[1][1].set_title("Štekot podľa citlivosti na hluk")
axes[1][1].set_xlabel("Skóre citlivosti na hluk")
axes[1][1].set_ylabel("Štekot")

plt.suptitle("Vzťah medzi citlivosťou na hluk a agresivitou/štekaním", fontsize=14)
plt.tight_layout()
plt.show()

### Teraz strach vs citlivost na hluk

In [ ]:
psy6 = dogs.copy(deep=True)
psy6["noise_sensitivity_score"] = psy6["noise_sensitivity_score"].round(1)

psy6 = psy6.groupby("noise_sensitivity_score")[["fearfulness_score"]].mean()

# Scatter plot: Strach vs. citlivosť na hluk
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
sns.scatterplot(x=psy6.index, y=psy6["fearfulness_score"], ax=ax)
ax.set_title("Strach vs. citlivosť na hluk")
ax.set_xlabel("Skóre citlivosti na hluk")
ax.set_ylabel("Skóre strachu")

# Environment

In [ ]:
# ['ownership_reason', 'obtained_from', 'age_enter_household', 'weaning_age', 'Socialization_men', 
#  'Socialization_women', 'Socialization_children', 'Socialization_dogs', 'Socialization_city', 
#  'Socialization_car', 'Socialization_public.transportation', 'socialization', 'socialization_quantile', 
#  'living_place', 'living_building', 'outdoor_access', 'other_dogs_when_obtained', 'currently_other_dogs', 
#  'daily_exercise', 'leashed_unleashed', 'trains_agility', 'trains_obedience_rally', 'trains_nose_work', 
#  'trains_rescue_utility', 'hobby_frequency_home', 'hobby_frequency_out', 'alone_time', 'life_experience', 
#  'dog_experience', 'dog_breeder', 'animal-related_work']

# ['Willing_to_learn', 'Persevering', 'Predictable', 'Obedient', 'Boisterous', 'Sensitive_touch', 
#  'Vocal', 'Anxious', 'Independent', 'Lazy', 'Intelligent', 'Reliable', 'Prey_driven_chase', 
#  'Human_dependent', 'Wary', 'Erratic', 'Stubborn', 'Fearful_people', 'Playful_people', 'Playful_alone', 
#  'Curious', 'Fearful_dogs', 'Impulsive', 'Affectionate_people', 'Decisive', 'Persevering', 'Distractible', 
#  'Submissive', 'Cautious', 'Restless', 'Confident', 'Easygoing', 'Dominant', 'Patient', 'Sociable_dogs_samegender', 
#  'Slow', 'Sociable_dogs_oppositegender', 'Calm', 'Provocative', 'Excitable', 'Human_oriented', 'Focused', 'Prey_driven_search', 
#  'Empathic', 'Aggressive_dogs_samegender', 'Greedy', 'Attention_seeking', 'Indifferent', 'Aggressive_people', 
#  'Affectionate_dogs', 'Vigilant', 'Calming', 'Solitary', 'Aggressive_dogs_oppositegender', 'Territorial', 'Flexible', 
#  'Easily_recovered', 'Bold', 'Active', 'Energetic', 'Insecure', 'Attentive', 'Sociable_people', 'insecurity_score', 
#  'training_focus_score', 'activity_playfulness_score', 'aggressiveness_dominance_score', 
#  'human_sociability_score', 'dog_sociability_score', 'perseverance_score']

order=['under_7_weeks', '7_weeks', '8_weeks', '9_weeks', 'over_9_weeks', 'still_with_dam',
       'unknown']

order=['under_1_hour', '1-2_hours', '2-3_hours', 'over_3_hours']

psy7 = dogs.copy(deep=True)
psy7 = psy7.groupby("daily_exercise").agg('mean', numeric_only=True)[personality]
psy7 = psy7.loc[order, :]
# display(psy7)

figure, ax = plt.subplots(1, 1, figsize=(6, 4), sharey=True)
sns.countplot(data=dogs, x="daily_exercise", ax=ax)
ax.tick_params(axis='x', rotation=25)

# daily_exercise
zaujimave = ['Fearful_dogs', 'Decisive', 'Playful_alone', 'Playful_people', 'Playful_dogs', 'Anxious', 'Lazy', 'Intelligent', 'Sociable_dogs_samegender']
dlzka = 6
figure, axes = plt.subplots(3, 3, figsize=(20, 20), sharex=True)
plt.xticks(ha='right') 
for i in range(9):
       sns.barplot(x=psy7.index, y=psy7[zaujimave[i]], ax=axes[i//3][i%3])
       axes[i//3][i%3].set_ylim(0, 5)
       axes[i//3][i%3].tick_params()


# +vsetky spojite vyzeraju zaujimavo











In [ ]:
psy8 = dogs.copy(deep=True)
psy8["owner_aggression_score"] = psy8["owner_aggression_score"].round(1)
psy8 = psy8.groupby("owner_aggression_score")[aggression_continuous+['Aggression_age']].mean()


# ax = sns.histplot(data=dogs, x="owner_aggression_score")

figure, axes = plt.subplots(1, 1, figsize=(6, 5), sharey=True)
sns.scatterplot(x=psy8.index, y=psy8["stranger_aggression_score"], data=psy8, ax=axes, hue='Aggression_age')

plt.tight_layout()
plt.show()

psy8 = dogs.copy(deep=True)
psy8["stranger_aggression_score"] = psy8["stranger_aggression_score"].round(1)
psy8 = psy8.groupby("stranger_aggression_score")[aggression_continuous+['Aggression_age']].mean()


# ax = sns.histplot(data=dogs, x="stranger_aggression_score")

figure, axes = plt.subplots(1, 1, figsize=(6, 5), sharey=True)
sns.scatterplot(x=psy8.index, y=psy8["owner_aggression_score"], data=psy8,  ax=axes, hue='Aggression_age')

plt.tight_layout()
plt.show()